In [3]:
#!pip install cv2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [1]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
%config Completer.use_jedi = False 

In [2]:
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
mask_detection = tf.keras.models.load_model("mask_new_detection.h5")

In [9]:
text_mask = "Mask"
text_no_mask= "Put your mask on!"
font=cv2.FONT_HERSHEY_TRIPLEX
scale= 0.6

In [10]:
def predict(image):
  face_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  face_frame = cv2.resize(face_frame, (224,224))
  face_frame = img_to_array(face_frame)
  face_frame = np.expand_dims(face_frame, axis=0)
  face_frame = preprocess_input(face_frame)
  prediction = mask_detection.predict(face_frame)
  return prediction[0][0]

In [11]:
def detector(gray_image, frame):
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5)
    
    for (x,y,w,h) in faces:
        roi_color = frame[y:y+h, x:x+w]
        mask = predict(roi_color)
        # Determine the label based on the mask prediction
        label = "Mask" if mask > 0.5 else "No Mask"
        # Format the label to include the confidence score
        label = "{}: {:.2f}%".format(label, max(mask, 1-mask) * 100)
        
        if mask > 0.5: #, text =text_mask
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2,lineType=cv2.LINE_AA)
            cv2.putText(frame,label, org =(x+50,y-100), fontFace =font, fontScale = scale, color =(0,255,0),thickness = 2)
        elif mask<=0.5: #text =text_no_mask,
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2,lineType=cv2.LINE_AA)
            cv2.putText(frame, label, org =(x+50,y-100), fontFace =font, fontScale = scale , color =(0,0,255),thickness = 2)
            
    return frame

In [14]:
video_cap = cv2.VideoCapture(0)

while True:
    ret, frame = video_cap.read()
  # Check if frame is successfully read
  #if not ret:
  #  break
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detect = detector(gray_frame, frame)
    cv2.imshow("Video",detect)

    if cv2.waitKey(1) & 0xFF == 27:
        break

video_cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 78ms/step


1/1 [==============================] - 0s 85ms/step


In [5]:
##############OLD CODE##############
def detector(gray_image, frame):
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5)
    
    for (x,y,w,h) in faces:
        roi_color = frame[y:y+h, x:x+w]
        mask = predict(roi_color)
       
        
        if mask > 0.5: #
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2,lineType=cv2.LINE_AA)
            cv2.putText(frame,text =text_mask, org =(x+50,y-100), fontFace =font, fontScale = scale, color =(0,255,0),thickness = 2)
        elif mask<=0.5: #
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2,lineType=cv2.LINE_AA)
            cv2.putText(frame,text =text_no_mask, org =(x+50,y-100), fontFace =font, fontScale = scale , color =(0,0,255),thickness = 2)
            
    return frame